In [125]:
# coding: utf-8
"""
ファイルの編集を色々とするためのファイル
"""
from filer2.filer2 import Filer
import numpy as np
import glob
import collections
import re
import MeCab
import itertools
import CaboCha
from stanford_corenlp_pywrapper import CoreNLP
import os
import csv

In [134]:
# 関数の定義
mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
proc = CoreNLP(configdict={'annotators': 'tokenize, ssplit, pos, lemma, parse'},
               corenlp_jars=["/home/ikegami/lib/stanford-corenlp-full-2015-04-20/*"])

def morpho(sentence):
    res = mecab.parseToNode(sentence)
    list_words = []
    while res:
        features = res.feature.split(",")
        if (features[0] == "名詞" and features[1] in ["一般", "固有名詞", "サ変接続", "形容動詞語幹"]) or features[0] == "形容詞":
            if features[6] == "*":
                if res.surface != '':
                    list_words.append(res.surface)
            else:
                if features[6] != '':
                    list_words.append(features[6])
        res = res.next
    return list_words

def morpho_v(sentence):
    res = mecab.parseToNode(sentence)
    list_words = []
    while res:
        features = res.feature.split(",")
        if (features[0] == "名詞" and features[1] in ["一般", "固有名詞", "サ変接続", "形容動詞語幹"]) or features[0] == "形容詞" or (features[0] == "動詞" and features[1] == "自立"):
            if features[6] == "*":
                if res.surface != '':
                    list_words.append(res.surface)
            else:
                if features[6] != '':
                    list_words.append(features[6])
        res = res.next
    return list_words

def morpho1(sentence):
    res = mecab.parseToNode(sentence)
    list_words = []
    while res:
        features = res.feature.split(",")
        if features[0] != "記号":
            if features[6] == "*":
                if res.surface != '':
                    list_words.append(res.surface)
            else:
                if features[6] != '':
                    list_words.append(features[6])
        res = res.next
    return list_words

cp = CaboCha.Parser('-f1 -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

def get_v_word(tree, chunk):
    surface = []
    for i in range(chunk.token_pos, chunk.token_pos + chunk.token_size):
        token = tree.token(i)
        features = token.feature.split(',')
        if features[0] == '名詞' and features[1] in ["一般", "固有名詞", "サ変接続", "形容動詞語幹"]:
            surface.append(token.surface)
        elif features[0] == '形容詞':
            surface.append(features[6])
        elif features[0] == '動詞' and features[1] == '自立':
            surface.append(features[6])
    return tuple(surface)

def get_v_words(line):
    cp = CaboCha.Parser('-f1 -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    tree = cp.parse(line)
    chunk_dic = {}
    chunk_id = 0
    for i in range(0, tree.size()):
        token = tree.token(i)
        if token.chunk:
            chunk_dic[chunk_id] = token.chunk
            chunk_id += 1

    tuples = []
    for chunk_id, chunk in chunk_dic.items():
        if chunk.link > 0:
            from_surface =  get_v_word(tree, chunk)
            to_chunk = chunk_dic[chunk.link]
            to_surface = get_v_word(tree, to_chunk)
            tuples.extend(itertools.product(from_surface, to_surface))
    return tuples

def get_word(tree, chunk):
    surface = []
    for i in range(chunk.token_pos, chunk.token_pos + chunk.token_size):
        token = tree.token(i)
        features = token.feature.split(',')
        if features[0] == '名詞' and features[1] in ["一般", "固有名詞", "サ変接続", "形容動詞語幹"]:
            surface.append(token.surface)
        elif features[0] == '形容詞':
            surface.append(features[6])
    return tuple(surface)

def get_words(line):
    cp = CaboCha.Parser('-f1 -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    tree = cp.parse(line)
    chunk_dic = {}
    chunk_id = 0
    for i in range(0, tree.size()):
        token = tree.token(i)
        if token.chunk:
            chunk_dic[chunk_id] = token.chunk
            chunk_id += 1

    tuples = []
    for chunk_id, chunk in chunk_dic.items():
        if chunk.link > 0:
            from_surface =  get_word(tree, chunk)
            to_chunk = chunk_dic[chunk.link]
            to_surface = get_word(tree, to_chunk)
            tuples.extend(itertools.product(from_surface, to_surface))
    return tuples

p_n = r'^NN'
p_v = r'^VB'
p_j = r'^JJ'

def morpho_e_njv(sentence):
    list_sentence = proc.parse_doc(sentence)['sentences']
    list_all_bag = []
    list_all_edge = []
    list_all_sentence = []
    list_all_lemmas = []
    for dict_tmp in list_sentence:
        sentence = [word for word in dict_tmp['tokens']
                    if word != ',' and word != '.' and word != ' ']
        list_all_sentence.append(' '.join(sentence))
        list_lemmas = dict_tmp['tokens']
        list_deps = dict_tmp['deps_cc']
        list_pos = dict_tmp['pos']
        dict_key_lemmas = {}
        dict_lemmas_key = {}
        list_bag = []
        for i, row in enumerate(zip(list_lemmas, list_pos)):
            lemmas = row[0]
            pos = row[1]

            if re.match(p_n, pos) or re.match(p_j, pos) or re.match(p_v, pos):
                dict_key_lemmas[i] = lemmas.encode('utf-8')
                dict_lemmas_key[lemmas.encode('utf-8')] = i
                list_bag.append(lemmas.encode('utf-8'))
            
        list_edge = []
        for deps in list_deps:
            if deps[1] in dict_key_lemmas and deps[2] in dict_key_lemmas:
                list_edge.append([dict_key_lemmas[deps[1]],
                                  dict_key_lemmas[deps[2]]])

        list_all_bag.append(list_bag)
        list_all_edge.append(list_edge)
    return list_all_sentence, list_all_bag, list_all_edge

def morpho_e(sentence):
    list_sentence = proc.parse_doc(sentence)['sentences']
    list_all_bag = []
    list_all_edge = []
    list_all_sentence = []
    for dict_tmp in list_sentence:
        sentence = [word for word in dict_tmp['tokens']
                    if word != ',' and word != '.' and word != ' ']
        list_all_sentence.append(' '.join(sentence))
        list_lemmas = dict_tmp['tokens']
        list_deps = dict_tmp['deps_cc']
        list_pos = dict_tmp['pos']
        dict_key_lemmas = {}
        dict_lemmas_key = {}
        list_bag = []
        for i, row in enumerate(zip(list_lemmas, list_pos)):
            lemmas = row[0]
            pos = row[1]

            if lemmas != ',' and lemmas != ' ' and lemmas != '.':
                dict_key_lemmas[i] = lemmas.encode('utf-8')
                dict_lemmas_key[lemmas.encode('utf-8')] = i
                list_bag.append(lemmas.encode('utf-8'))
            
        list_edge = []
        for deps in list_deps:
            if deps[1] in dict_key_lemmas and deps[2] in dict_key_lemmas:
                list_edge.append([dict_key_lemmas[deps[1]],
                                  dict_key_lemmas[deps[2]]])

        list_all_bag.append(list_bag)
        list_all_edge.append(list_edge)
    return list_all_sentence, list_all_bag, list_all_edge

def morpho_e_nj(sentence):
    list_sentence = proc.parse_doc(sentence)['sentences']
    list_all_bag = []
    list_all_edge = []
    list_all_sentence = []
    for dict_tmp in list_sentence:
        sentence = [word for word in dict_tmp['tokens']
                    if word != ',' and word != '.' and word != ' ']
        list_all_sentence.append(' '.join(sentence))
        list_lemmas = dict_tmp['tokens']
        list_deps = dict_tmp['deps_cc']
        list_pos = dict_tmp['pos']
        dict_key_lemmas = {}
        dict_lemmas_key = {}
        list_bag = []
        for i, row in enumerate(zip(list_lemmas, list_pos)):
            lemmas = row[0]
            pos = row[1]

            if re.match(p_n, pos) or re.match(p_j, pos):
                dict_key_lemmas[i] = lemmas.encode('utf-8')
                dict_lemmas_key[lemmas.encode('utf-8')] = i
                list_bag.append(lemmas.encode('utf-8'))
            
        list_edge = []
        for deps in list_deps:
            if deps[1] in dict_key_lemmas and deps[2] in dict_key_lemmas:
                list_edge.append([dict_key_lemmas[deps[1]],
                                  dict_key_lemmas[deps[2]]])

        list_all_bag.append(list_bag)
        list_all_edge.append(list_edge)
    return list_all_sentence, list_all_bag, list_all_edge


def morpho_e_njv_pos(sentence):
    list_sentence = proc.parse_doc(sentence)['sentences']
    list_all_bag = []
    list_all_edge = []
    list_all_sentence = []
    list_all_lemmas = []
    list_all_pos = []
    for dict_tmp in list_sentence:
        sentence = [word for word in dict_tmp['tokens']
                    if word != ',' and word != '.' and word != ' ']
        list_all_sentence.append(' '.join(sentence))
        list_lemmas = dict_tmp['tokens']
        list_deps = dict_tmp['deps_cc']
        list_pos = dict_tmp['pos']
        dict_key_lemmas = {}
        dict_lemmas_key = {}
        list_bag = []
        list_p = []
        for i, row in enumerate(zip(list_lemmas, list_pos)):
            lemmas = row[0]
            pos = row[1]

            if re.match(p_n, pos):
                dict_key_lemmas[i] = lemmas.encode('utf-8')
                dict_lemmas_key[lemmas.encode('utf-8')] = i
                list_bag.append(lemmas.encode('utf-8'))
                list_p.append('n')
            elif re.match(p_j, pos):
                dict_key_lemmas[i] = lemmas.encode('utf-8')
                dict_lemmas_key[lemmas.encode('utf-8')] = i
                list_bag.append(lemmas.encode('utf-8'))
                list_p.append('j')
            elif re.match(p_v, pos):
                dict_key_lemmas[i] = lemmas.encode('utf-8')
                dict_lemmas_key[lemmas.encode('utf-8')] = i
                list_bag.append(lemmas.encode('utf-8'))
                list_p.append('v')
            
        list_edge = []
        for deps in list_deps:
            if deps[1] in dict_key_lemmas and deps[2] in dict_key_lemmas:
                list_edge.append([dict_key_lemmas[deps[1]],
                                  dict_key_lemmas[deps[2]]])

        list_all_bag.append(list_bag)
        list_all_edge.append(list_edge)
        list_all_pos.append(list_p)
    return list_all_sentence, list_all_bag, list_all_edge, list_all_pos

def morpho_e_pos(sentence):
    list_sentence = proc.parse_doc(sentence)['sentences']
    list_all_bag = []
    list_all_edge = []
    list_all_sentence = []
    list_all_pos = []
    for dict_tmp in list_sentence:
        sentence = [word for word in dict_tmp['tokens']
                    if word != ',' and word != '.' and word != ' ']
        list_all_sentence.append(' '.join(sentence))
        list_lemmas = dict_tmp['tokens']
        list_deps = dict_tmp['deps_cc']
        list_pos = dict_tmp['pos']
        dict_key_lemmas = {}
        dict_lemmas_key = {}
        list_bag = []
        list_p = []
        for i, row in enumerate(zip(list_lemmas, list_pos)):
            lemmas = row[0]
            pos = row[1]

            if lemmas != ',' and lemmas != ' ' and lemmas != '.':
                dict_key_lemmas[i] = lemmas.encode('utf-8')
                dict_lemmas_key[lemmas.encode('utf-8')] = i
                list_bag.append(lemmas.encode('utf-8'))
                if re.match(p_n, pos):
                    list_p.append('n')
                elif re.match(p_j, pos):
                    list_p.append('j')
                elif re.match(p_v, pos):
                    list_p.append('v')
                else:
                    list_p.append('-')
            
        list_edge = []
        for deps in list_deps:
            if deps[1] in dict_key_lemmas and deps[2] in dict_key_lemmas:
                list_edge.append([dict_key_lemmas[deps[1]],
                                  dict_key_lemmas[deps[2]]])

        list_all_bag.append(list_bag)
        list_all_edge.append(list_edge)
        list_all_pos.append(list_p)
    return list_all_sentence, list_all_bag, list_all_edge, list_all_pos

def morpho_e_nj_pos(sentence):
    list_sentence = proc.parse_doc(sentence)['sentences']
    list_all_bag = []
    list_all_edge = []
    list_all_sentence = []
    list_all_pos = []
    for dict_tmp in list_sentence:
        sentence = [word for word in dict_tmp['tokens']
                    if word != ',' and word != '.' and word != ' ']
        list_all_sentence.append(' '.join(sentence))
        list_lemmas = dict_tmp['tokens']
        list_deps = dict_tmp['deps_cc']
        list_pos = dict_tmp['pos']
        dict_key_lemmas = {}
        dict_lemmas_key = {}
        list_bag = []
        list_p = []
        for i, row in enumerate(zip(list_lemmas, list_pos)):
            lemmas = row[0]
            pos = row[1]

            if re.match(p_n, pos):
                dict_key_lemmas[i] = lemmas.encode('utf-8')
                dict_lemmas_key[lemmas.encode('utf-8')] = i
                list_bag.append(lemmas.encode('utf-8'))
                list_p.append('n')
            elif re.match(p_j, pos):
                dict_key_lemmas[i] = lemmas.encode('utf-8')
                dict_lemmas_key[lemmas.encode('utf-8')] = i
                list_bag.append(lemmas.encode('utf-8'))
                list_p.append('j')
            
        list_edge = []
        for deps in list_deps:
            if deps[1] in dict_key_lemmas and deps[2] in dict_key_lemmas:
                list_edge.append([dict_key_lemmas[deps[1]],
                                  dict_key_lemmas[deps[2]]])

        list_all_bag.append(list_bag)
        list_all_edge.append(list_edge)
        list_all_pos.append(list_p)
    return list_all_sentence, list_all_bag, list_all_edge, list_all_pos

def morpho_j(sentence):
    res = mecab.parseToNode(sentence)
    list_words = []
    list_all_word = []
    list_all_lemmas = []
    list_lemmas = []
    while res:
        features = res.feature.split(",")
        pos = features[0]
        subpos = features[1]
        tokens = res.surface
        lemmas = features[6]
        if subpos != "読点" and subpos != "句点" and tokens != '':
            list_all_word.append(tokens)
            if lemmas != '*' and pos != '名詞':
                list_all_lemmas.append(lemmas)
            else:
                list_all_lemmas.append(tokens)
        if (pos == "名詞" and subpos in ["一般", "固有名詞", "サ変接続", "形容動詞語幹"]) or pos == "形容詞":
            list_words.append(tokens)
            if lemmas != '*' and pos != '名詞':
                list_lemmas.append(lemmas)
            else:
                list_lemmas.append(tokens)
        res = res.next
    return list_all_word, list_words, list_all_lemmas, list_lemmas

INFO:CoreNLP_PyWrapper:Starting java subprocess, and waiting for signal it's ready, with command: exec java -Xmx4g -XX:ParallelGCThreads=1 -cp '/home/ikegami/ikegami/lib/python2.7/site-packages/stanford_corenlp_pywrapper/lib/*:/home/ikegami/lib/stanford-corenlp-full-2015-04-20/*'      corenlp.SocketServer --outpipe /tmp/corenlp_pywrap_pipe_pypid=12464_time=1475136941.89  --configdict '{"annotators": "tokenize, ssplit, pos, lemma, parse"}'
INFO:CoreNLP_PyWrapper:Successful ping. The server has started.
INFO:CoreNLP_PyWrapper:Subprocess is ready.


In [92]:
list_sentence, list_lemmas, list_bag, list_edge = morpho_e('The clusters of review sentences on the viewpoints from the products evaluation can be applied to various use. The topic models, for example Unigram Mixture (UM), can be used for this task.')

In [93]:
for sentence, lemmas, bag, edge in zip(list_sentence, list_lemmas, list_bag, list_edge):
    print sentence, lemmas, bag, edge

[u'The', u'clusters', u'of', u'review', u'sentences', u'on', u'the', u'viewpoints', u'from', u'the', u'products', u'evaluation', u'can', u'be', u'applied', u'to', u'various', u'use'] [u'the', u'cluster', u'of', u'review', u'sentence', u'on', u'the', u'viewpoint', u'from', u'the', u'product', u'evaluation', u'can', u'be', u'apply', u'to', u'various', u'use'] ['cluster', 'review', 'sentence', 'viewpoint', 'product', 'evaluation', 'various', 'use'] [['cluster', 'sentence'], ['sentence', 'review'], ['sentence', 'viewpoint'], ['viewpoint', 'evaluation'], ['evaluation', 'product'], ['use', 'various']]
[u'The', u'topic', u'models', u'for', u'example', u'Unigram', u'Mixture', u'-LRB-', u'UM', u'-RRB-', u'can', u'be', u'used', u'for', u'this', u'task'] [u'the', u'topic', u'model', u'for', u'example', u'Unigram', u'Mixture', u'-lrb-', u'UM', u'-rrb-', u'can', u'be', u'use', u'for', u'this', u'task'] ['topic', 'model', 'example', 'Unigram', 'Mixture', 'UM', 'task'] [['model', 'Mixture'], ['model'

### 全体のコーパスを施設別に集計する

In [4]:
list_filepath = glob.glob('../topic_model/files/rakuten_corpus/rakuten_corpus_master/txtfile/*')

In [5]:
# 読み込み
list_file = [Filer.readtxt(path)  for path in list_filepath]

In [12]:
# タブで区切る
list_file = [[sentence.split('\t') for sentence in row] for row in list_file]

In [53]:
# ホテル番号別に集計
dict_hotel_sentence = collections.defaultdict(list)
for row in list_file:
    for sen in row:
        dict_hotel_sentence[sen[0]].append(sen[1] + '\t' + sen[2])

In [54]:
# review数が100件未満のホテルは除外して保存する
for key, value in dict_hotel_sentence.items():
    if len(value) >= 100:
        Filer.writetxt(value, '../files/txtfile/review-'+key+'.txt')

In [87]:
# 数を集計
dict_count = {key: len(value) for key, value in dict_hotel_sentence.items()}

In [50]:
# 施設名と番号を合わせる
list_name = Filer.readtxt('../files/travel03_hotelMaster_20160304.txt')
list_name = [sen.split('\t') for sen in list_name]
dict_num_name = {row[0]: row[1] for row in list_name if len(row) == 2}

In [86]:
# 年別にレビュー数を集計
dict_counter = {}
for key, value in dict_hotel_sentence.items():
    dict_counter_tmp = collections.defaultdict(int)
    for sen in value:
        dict_counter_tmp[sen[0:4]] += 1
    dict_counter[key] = dict_counter_tmp

In [84]:
dict_count['9680']

defaultdict(int,
            {'2001': 3,
             '2002': 6,
             '2003': 17,
             '2004': 6,
             '2005': 54,
             '2006': 288,
             '2007': 96,
             '2008': 42,
             '2009': 38,
             '2010': 31,
             '2011': 11})

In [90]:
# 施設番号、施設名、レビュー数のファイルを作成する
list_master = [['施設番号', '施設名', 'レビュー数',
                '1999', '2000', '2001', '2002',
                '2003', '2004', '2005', '2006',
                '2007', '2008', '2009', '2010',
                '2011']]
for key, value in sorted(dict_count.items(), key=lambda x:x[1], reverse=True):
    if key in dict_num_name:
        list_master.append([key, dict_num_name[key], value, dict_counter[key]['1999'],
                            dict_counter[key]['2000'], dict_counter[key]['2001'], dict_counter[key]['2002'],
                            dict_counter[key]['2003'], dict_counter[key]['2004'], dict_counter[key]['2005'],
                            dict_counter[key]['2006'], dict_counter[key]['2007'], dict_counter[key]['2008'],
                            dict_counter[key]['2009'], dict_counter[key]['2010'], dict_counter[key]['2011']])
    else:
        list_master.append([key, '', value, dict_counter[key]['1999'],
                            dict_counter[key]['2000'], dict_counter[key]['2001'], dict_counter[key]['2002'],
                            dict_counter[key]['2003'], dict_counter[key]['2004'], dict_counter[key]['2005'],
                            dict_counter[key]['2006'], dict_counter[key]['2007'], dict_counter[key]['2008'],
                            dict_counter[key]['2009'], dict_counter[key]['2010'], dict_counter[key]['2011']])

In [91]:
# csvfileとして保存
Filer.writecsv(list_master, '../files/txtfile/master.csv')

### 分析対象とするホテルのみを取り出して、sentenceごとのファイルにする
* 51637：アパホテル＆リゾート＜東京ベイ幕張＞
* 1238：オリエンタルホテル東京ベイ
* 53171：加賀の湧泉　ドーミーイン金沢
* 19191：新横浜プリンスホテル
* 76887：ホテルモントレ　グラスミア大阪

In [92]:
list_filepath = ['../files/txtfile/review-51637.txt',
                 '../files/txtfile/review-1238.txt',
                 '../files/txtfile/review-53171.txt',
                 '../files/txtfile/review-19191.txt',
                 '../files/txtfile/review-76887.txt',]

In [93]:
# 2010年、2011年の文章のみを取得
list_review_rev = []
for path in list_filepath:
    list_review = Filer.readtxt(path)
    list_tmp = []
    for review in list_review:
        if review[0:4] == '2010' or review[0:4] == '2011':
            list_tmp.append(review.split('\t')[1])
    list_review_rev.append(list_tmp)

In [103]:
#【ご利用の宿泊プラン】~を削除
pattern = r'【ご利用の宿泊プラン】.+$'
list_review_rev1 = []
for row in list_review_rev:
    list_tmp = []
    for sentence in row:
        list_tmp.append(re.sub(pattern, '', sentence))
    list_review_rev1.append(list_tmp)

In [164]:
# sentenceに変更、その際、ユーザー番号を頭に振る
list_sentence = []
for row in list_review_rev1:
    list_tmp = []
    for i, review in enumerate(row):
        list_s = re.split(r'。|！|？', review)
        for sen in list_s:
            list_tmp.append([i, sen])
    list_sentence.append(list_tmp)

In [165]:
# 文字の前後の空白を削除
list_sentence_rev = []
for row in list_sentence:
    list_tmp = []
    for sen in row:
        sentence = sen[1].replace(' ', '')
        if len(sentence) > 1:
            list_tmp.append([sen[0], sentence])
    list_sentence_rev.append(list_tmp)

In [185]:
# 先頭が、」もしくは）で始まってる場合、前の文にくっつける
list_sentence_rev1 = []
for row in list_sentence_rev:
    list_tmp = []
    for i in range(len(row)-1):
        if re.match(r'」|』|\)|）', row[i][1]):
            pass
        elif re.match(r'」|』|\)|）', row[i+1][1]):
            list_tmp.append([row[i][0], row[i][1]+row[i+1][1]])
        else:
            list_tmp.append([row[i][0], row[i][1]])
    else:
        list_tmp.append([row[i+1][0], row[i+1][1]])
        list_sentence_rev1.append(list_tmp)

In [187]:
Filer.writetsv(list_sentence_rev1[0], '../files/sentencefile/sentence-51637.tsv')
Filer.writetsv(list_sentence_rev1[1], '../files/sentencefile/sentence-1238.tsv')
Filer.writetsv(list_sentence_rev1[2], '../files/sentencefile/sentence-53171.tsv')
Filer.writetsv(list_sentence_rev1[3], '../files/sentencefile/sentence-19191.tsv')
Filer.writetsv(list_sentence_rev1[4], '../files/sentencefile/sentence-76887.tsv')

### sentenceごとのファイルを形態素解析済みのファイルに変更する

In [188]:
list_path = glob.glob('../files/sentencefile/*')
print list_path

['../files/sentencefile/sentence-76887.tsv', '../files/sentencefile/sentence-1238.tsv', '../files/sentencefile/sentence-53171.tsv', '../files/sentencefile/sentence-19191.tsv', '../files/sentencefile/sentence-51637.tsv']


In [189]:
list_sentence = [Filer.readtsv(path) for path in list_path]

In [198]:
list_morpho = [[[sen[1], morpho(sen[1])] for sen in row]for row in list_sentence]

In [209]:
Filer.writedump(list_morpho[4], '../files/morphofile/type1/morpho-51637.dump')

In [210]:
list_morpho1 = [[[sen[1], morpho1(sen[1])] for sen in row] for row in list_sentence]

In [216]:
Filer.writedump(list_morpho1[4], '../files/morphofile/type2/morpho-51637.dump')

### 高頻度語、低頻度語を数え上げて削除したファイルを作成する

In [218]:
list_word = [word for row in list_morpho for sen in row for word in sen[1]]
list_word1 = [word for row in list_morpho1 for sen in row for word in sen[1]]
dict_word_freq = collections.Counter(list_word)
dict_word_freq1 = collections.Counter(list_word1)
list_set_word = []
for row in list_morpho:
    for sen in row:
        list_set_word.extend(list(set(sen[1])))
        
list_set_word1 = []
for row in list_morpho1:
    for sen in row:
        list_set_word1.extend(list(set(sen[1])))

dict_document_freq = collections.Counter(list_set_word)
dict_document_freq1 = collections.Counter(list_set_word1)
print len(list_morpho[0])+len(list_morpho[1])+len(list_morpho[2])+len(list_morpho[3])+len(list_morpho[4])
print len(list_morpho1[0])+len(list_morpho1[1])+len(list_morpho1[2])+len(list_morpho1[3])+len(list_morpho1[4])

In [244]:
# 頻度の確認、10%切りで良さそう
for row in sorted(dict_document_freq.items(), key=lambda x:x[1], reverse=True)[0:10]:
    print row[0], row[1]

部屋 0.166852010265
利用 0.136783575706
ホテル 0.100427715997
良い 0.0860992301112
宿泊 0.0706159110351
満足 0.0563301967494
便利 0.047005988024
朝食 0.0458083832335
よい 0.0455089820359
広い 0.0444824636441


In [245]:
# 除去語の記録、10%以上、2回以下
list_remove = []
for row in dict_document_freq.items():
    if row[1] >= 0.1:
        list_remove.append(row[0])
for row in dict_word_freq.items():
    if row[1] <= 2:
        list_remove.append(row[0])

In [262]:
Filer.writetxt(list_remove, '../files/preprocessedfile/dict/remove_words.txt')

In [249]:
list_path = glob.glob('../files/morphofile/type1/*.dump')
list_path

['../files/morphofile/type1/morpho-51637.dump',
 '../files/morphofile/type1/morpho-53171.dump',
 '../files/morphofile/type1/morpho-1238.dump',
 '../files/morphofile/type1/morpho-76887.dump',
 '../files/morphofile/type1/morpho-19191.dump']

In [250]:
list_sentence = [Filer.readdump(path) for path in list_path]
list_sentence_rev = []
for row in list_sentence:
    list_tmp = []
    for sen in row:
        list_tmp_tmp = [word for word in sen[1] if word not in list_remove]
        list_tmp.append([sen[0], list_tmp_tmp])
    list_sentence_rev.append(list_tmp)

In [258]:
# 総単語数が１以下の文を削除
list_sentence_rev = [[sen for sen in row if len(sen[1]) >= 2] for row in list_sentence_rev]

In [267]:
Filer.writedump(list_sentence_rev[4], '../files/preprocessedfile/type11/wordlist/preprocessed-19191.dump')

### エッジリストの作成

In [268]:
list_path = glob.glob('../files/preprocessedfile/type11/wordlist/*')
list_path

['../files/preprocessedfile/type11/wordlist/preprocessed-19191.dump',
 '../files/preprocessedfile/type11/wordlist/preprocessed-76887.dump',
 '../files/preprocessedfile/type11/wordlist/preprocessed-1238.dump',
 '../files/preprocessedfile/type11/wordlist/preprocessed-51637.dump',
 '../files/preprocessedfile/type11/wordlist/preprocessed-53171.dump']

In [269]:
list_sentence = [Filer.readdump(path) for path in list_path]

In [270]:
# bigram
list_bigram = []
for row in list_sentence:
    list_tmp = []
    for sen in row:
        for i in range(len(sen[1])-1):
            list_tmp.append([sen[1][i], sen[1][i+1]])
    list_bigram.append(list_tmp)

In [279]:
Filer.writetsv(list_bigram[4], '../files/preprocessedfile/type11/bigram/bigram-53171.tsv')

In [322]:
# cor
list_cor = []
for row in list_sentence:
    list_tmp = []
    for sen in row:
        list_tmp.extend(list(itertools.combinations(tuple(sen[1]),2)))
    list_tmp = [list(row) for row in list_tmp]
    list_cor.append(list_tmp)

In [330]:
Filer.writetsv(list_cor[4], '../files/preprocessedfile/type11/cor/cor-53171.tsv')

In [311]:
seq = ['a', 'b', 'c', 'd', 'e']
list(itertools.combinations(tuple(seq),2))

[('a', 'b'),
 ('a', 'c'),
 ('a', 'd'),
 ('a', 'e'),
 ('b', 'c'),
 ('b', 'd'),
 ('b', 'e'),
 ('c', 'd'),
 ('c', 'e'),
 ('d', 'e')]

### PRTMによって、分類した後の文書を形態素解析する

In [30]:
list_path = glob.glob('./files/classifiedfile/type11/bigram/sentence/*')
removeword = './files/classifiedfile/type11/bigram/sentence/classified'
list_name = [path.replace(removeword, '') for path in list_path]
list_name = [name.replace('.txt', '') for name in list_name]

In [31]:
# 読み込み
list_sentence = [Filer.readtxt(path) for path in list_path]

In [33]:
# 形態素解析
list_master = []
for row in list_sentence:
    list_tmp = [morpho_v(sentence) for sentence in row]
    list_master.append(list_tmp)

In [34]:
# 保存
for row, name in zip(list_master, list_name):
    Filer.writetsv(row, './files/classifiedfile/type11/bigram/morpho/n_adj_verb/classified'+name+'.tsv')

In [25]:
# エッジリストの作成
list_bigram = []
for row in list_master:
    list_tmp = []
    for sen in row:
        for i in range(len(sen)-1):
            list_tmp.append([sen[i], sen[i+1]])
    list_bigram.append(list_tmp)

In [28]:
# 保存
for row, name in zip(list_bigram, list_name):
    Filer.writetsv(row, './classifiedfile/type11/bigram/bigram/classified'+name+'.tsv')

In [29]:
# cor
list_cor = []
for row in list_master:
    list_tmp = []
    for sen in row:
        list_tmp.extend(list(itertools.combinations(tuple(sen),2)))
    list_tmp = [list(row) for row in list_tmp]
    list_cor.append(list_tmp)

In [33]:
# 保存
for row, name in zip(list_cor, list_name):
    Filer.writetsv(row, './classifiedfile/type11/bigram/cor/classified'+name+'.tsv')

### 構文構造を利用してエッジリストを作成

In [51]:
list_path = glob.glob('files/classifiedfile/type11/bigram/sentence/classified*')
removeword = 'files/classifiedfile/type11/bigram/sentence/classified'
list_name = [path.replace(removeword, '') for path in list_path]
list_name = [name.replace('.txt', '') for name in list_name]

In [52]:
# 読み込み
list_sentence = [Filer.readtxt(path) for path in list_path]

In [53]:
# 構文によるエッジリストの作成(動詞なし)
list_master = []
for row in list_sentence:
    list_tmp = [[t[0], t[1]] for sentence in row for t in get_words(sentence)]
    list_master.append(list_tmp)

In [55]:
for row in list_master[0][0:10]:
    print row[0], '=>', row[1]

開業 => 新横浜
開業 => プリンス
駅 => 近い
近い => 近い
アリーナ => 近い
近い => 利用
新横浜 => イベント
横浜アリーナ => ライヴ
ライヴ => あり
ライヴ => 利用


In [56]:
# 保存
for row, name in zip(list_master, list_name):
    Filer.writetsv(row, './files/classifiedfile/type11/bigram/syntax/non_verb/classified'+name+'.tsv')

In [57]:
# 構文によるエッジリストの作成(動詞あり)
list_master = []
for row in list_sentence:
    list_tmp = [[t[0], t[1]] for sentence in row for t in get_v_words(sentence)]
    list_master.append(list_tmp)

In [58]:
# 保存
for row, name in zip(list_master, list_name):
    Filer.writetsv(row, './files/classifiedfile/type11/bigram/syntax/verb/classified'+name+'.tsv')

In [50]:
sentence = '時間的に混むのがわかったいるんだから、もう少しフロントに人を配置して欲しい'
t = get_words(sentence)
for row in t:
    print row[0], ' => ', row[1]

フロント  =>  配置
フロント  =>  欲しい
人  =>  配置
人  =>  欲しい


### OpinosisDatasetの基礎集計

In [82]:
list_path_corpus = glob.glob('./files/OpinosisDataset1.0/topics/*')
list_path_gold = glob.glob('./files/OpinosisDataset1.0/summaries-gold/*')
print len(list_path_corpus)
print len(list_path_gold)

51
51


In [86]:
list_corpus_gold = Filer.readtxt('./files/OpinosisDataset1.0/summaries-gold/accuracy_garmin_nuvi_255W_gps/accuracy_garmin_nuvi_255W_gps.2.gold', LF='\r\n')

['The Garmin seems to be generally very accurate.',
 "It's easy to use with an intuitive interface."]

### OpinosisDatasetの形態素解析
1. トピックごとにdictを作成する
 * sentence: 元の文のリスト
 * sep_all: 含まれているすべての単語のリスト
 * sep_nj: 含まれている名詞と形容詞のリスト
 * sep_njv: 含まれている名詞と形容詞と動詞のリスト
 * edge_nj: すべての単語の構文によるエッジ
 * edge_nj: 名詞と形容詞の構文によるエッジ
 * edge_njv: 名詞と形容詞と動詞の構文によるエッジ

In [4]:
list_path = glob.glob('./files/OpinosisDataset1.0/topics/*')

In [5]:
proc = CoreNLP(configdict={'annotators': 'tokenize, ssplit, pos, lemma, parse'},
               corenlp_jars=["/home/ikegami/lib/stanford-corenlp-full-2015-04-20/*"])

for path in list_path:
    dict_total = {'sentence':[],
                  'sep_all':[],
                  'sep_nj':[],
                  'sep_njv':[],
                  'edge_all': [],
                  'edge_nj':[],
                  'edge_njv': []}
    filename = path.replace('./files/OpinosisDataset1.0/topics/', '').replace('.txt.data', '')
    outputpath = './files/OpinosisDataset1.0/preprocessed1/'+filename+'.dump'
    list_corpus = Filer.readtxt(path, LF='\r\n')
    for corpus in list_corpus:
        try:
            list_sentence_a, list_bag_a, list_edge_a = morpho_e(corpus)
            list_sentence_e, list_bag_e, list_edge_e = morpho_e_nj(corpus)
            list_sentence_v, list_bag_v, list_edge_v = morpho_e_njv(corpus)
            dict_total['sentence'].extend(list_sentence_e)
            dict_total['sep_all'].extend(list_bag_a)
            dict_total['sep_nj'].extend(list_bag_e)
            dict_total['sep_njv'].extend(list_bag_v)
            dict_total['edge_all'].extend(list_edge_a)
            dict_total['edge_nj'].extend(list_edge_e)
            dict_total['edge_njv'].extend(list_edge_v)
        except UnicodeDecodeError:
            print 'error'
    Filer.writedump(dict_total, outputpath)
    print outputpath

INFO:CoreNLP_PyWrapper:Starting java subprocess, and waiting for signal it's ready, with command: exec java -Xmx4g -XX:ParallelGCThreads=1 -cp '/home/ikegami/ikegami/lib/python2.7/site-packages/stanford_corenlp_pywrapper/lib/*:/home/ikegami/lib/stanford-corenlp-full-2015-04-20/*'      corenlp.SocketServer --outpipe /tmp/corenlp_pywrap_pipe_pypid=12569_time=1472178902.01  --configdict '{"annotators": "tokenize, ssplit, pos, lemma, parse"}'
INFO:CoreNLP_PyWrapper:Successful ping. The server has started.
INFO:CoreNLP_PyWrapper:Subprocess is ready.


./files/OpinosisDataset1.0/preprocessed1/gas_mileage_toyota_camry_2007.dump
error
error
error
error
error
./files/OpinosisDataset1.0/preprocessed1/location_bestwestern_hotel_sfo.dump
./files/OpinosisDataset1.0/preprocessed1/interior_honda_accord_2008.dump
./files/OpinosisDataset1.0/preprocessed1/sound_ipod_nano_8gb.dump
./files/OpinosisDataset1.0/preprocessed1/size_asus_netbook_1005ha.dump
./files/OpinosisDataset1.0/preprocessed1/eyesight-issues_amazon_kindle.dump
./files/OpinosisDataset1.0/preprocessed1/display_garmin_nuvi_255W_gps.dump
./files/OpinosisDataset1.0/preprocessed1/video_ipod_nano_8gb.dump
error
./files/OpinosisDataset1.0/preprocessed1/buttons_amazon_kindle.dump
./files/OpinosisDataset1.0/preprocessed1/screen_ipod_nano_8gb.dump
error
./files/OpinosisDataset1.0/preprocessed1/location_holiday_inn_london.dump
./files/OpinosisDataset1.0/preprocessed1/comfort_honda_accord_2008.dump
error
./files/OpinosisDataset1.0/preprocessed1/service_bestwestern_hotel_sfo.dump
./files/Opinosi

### OpinosisDataset.goldの形態素解析

In [6]:
list_path = glob.glob('./files/OpinosisDataset1.0/summaries-gold/*')

In [7]:
list_path

['./files/OpinosisDataset1.0/summaries-gold/bathroom_bestwestern_hotel_sfo',
 './files/OpinosisDataset1.0/summaries-gold/staff_swissotel_chicago',
 './files/OpinosisDataset1.0/summaries-gold/navigation_amazon_kindle',
 './files/OpinosisDataset1.0/summaries-gold/performance_netbook_1005ha',
 './files/OpinosisDataset1.0/summaries-gold/free_bestwestern_hotel_sfo',
 './files/OpinosisDataset1.0/summaries-gold/speed_windows7',
 './files/OpinosisDataset1.0/summaries-gold/eyesight-issues_amazon_kindle',
 './files/OpinosisDataset1.0/summaries-gold/staff_bestwestern_hotel_sfo',
 './files/OpinosisDataset1.0/summaries-gold/sound_ipod_nano_8gb',
 './files/OpinosisDataset1.0/summaries-gold/screen_netbook_1005ha',
 './files/OpinosisDataset1.0/summaries-gold/features_windows7',
 './files/OpinosisDataset1.0/summaries-gold/fonts_amazon_kindle',
 './files/OpinosisDataset1.0/summaries-gold/interior_toyota_camry_2007',
 './files/OpinosisDataset1.0/summaries-gold/performance_honda_accord_2008',
 './files/Op

In [8]:
proc = CoreNLP(configdict={'annotators': 'tokenize, ssplit, pos, lemma, parse'},
               corenlp_jars=["/home/ikegami/lib/stanford-corenlp-full-2015-04-20/*"])

for path in list_path:
    filename = path.replace('./files/OpinosisDataset1.0/summaries-gold/', '')
    list_filepath = glob.glob(path+'/*')
    dict_total = {'sentence':[],
                  'sep_all':[]}
    for path1 in list_filepath:
        list_sentence = []
        list_bag = []
        list_text = Filer.readtxt(path1, LF='\r\n')
        for sentence in list_text:
            list_sentence_a, list_bag_a, _ = morpho_e(sentence)
            list_sentence.extend(list_sentence_a)
            list_bag.extend(list_bag_a)
        dict_total['sentence'].append(list_sentence)
        dict_total['sep_all'].append(list_bag)
    print filename
    Filer.writedump(dict_total, './files/OpinosisDataset1.0/preprocessed1/ans/'+filename+'.dump')

INFO:CoreNLP_PyWrapper:Starting java subprocess, and waiting for signal it's ready, with command: exec java -Xmx4g -XX:ParallelGCThreads=1 -cp '/home/ikegami/ikegami/lib/python2.7/site-packages/stanford_corenlp_pywrapper/lib/*:/home/ikegami/lib/stanford-corenlp-full-2015-04-20/*'      corenlp.SocketServer --outpipe /tmp/corenlp_pywrap_pipe_pypid=12569_time=1472184282.29  --configdict '{"annotators": "tokenize, ssplit, pos, lemma, parse"}'
INFO:CoreNLP_PyWrapper:Successful ping. The server has started.
INFO:CoreNLP_PyWrapper:Subprocess is ready.


bathroom_bestwestern_hotel_sfo
staff_swissotel_chicago
navigation_amazon_kindle
performance_netbook_1005ha
free_bestwestern_hotel_sfo
speed_windows7
eyesight-issues_amazon_kindle
staff_bestwestern_hotel_sfo
sound_ipod_nano_8gb
screen_netbook_1005ha
features_windows7
fonts_amazon_kindle
interior_toyota_camry_2007
performance_honda_accord_2008
service_bestwestern_hotel_sfo
location_holiday_inn_london
service_swissotel_hotel_chicago
directions_garmin_nuvi_255W_gps
buttons_amazon_kindle
price_holiday_inn_london
food_holiday_inn_london
battery-life_amazon_kindle
location_bestwestern_hotel_sfo
updates_garmin_nuvi_255W_gps
accuracy_garmin_nuvi_255W_gps
battery-life_ipod_nano_8gb
display_garmin_nuvi_255W_gps
food_swissotel_chicago
voice_garmin_nuvi_255W_gps
transmission_toyota_camry_2007
rooms_bestwestern_hotel_sfo
keyboard_netbook_1005ha
seats_honda_accord_2008
gas_mileage_toyota_camry_2007
parking_bestwestern_hotel_sfo
screen_ipod_nano_8gb
interior_honda_accord_2008
size_asus_netbook_1005ha


In [9]:
dict_file = Filer.readdump('./files/OpinosisDataset1.0/preprocessed1/test/accuracy_garmin_nuvi_255W_gps.dump')

In [34]:
filepath = glob.glob('./files/OpinosisDataset1.0/preprocessed/test/*')
for path in filepath:
    print path
    dict_tmp = Filer.readdump(path)
    list_word = set([word for row in dict_tmp['sep_all'] for word in row])
    list_edge = set([word for row in dict_tmp['edge_all'] for words in row for word in words])
    print 'all', len(list_word), len(list_edge), list_edge.issubset(list_word)
    list_word = set([word for row in dict_tmp['sep_nj'] for word in row])
    list_edge = set([word for row in dict_tmp['edge_nj'] for words in row for word in words])
    print 'nj', len(list_word), len(list_edge), list_edge.issubset(list_word)
    list_word = set([word for row in dict_tmp['sep_njv'] for word in row])
    list_edge = set([word for row in dict_tmp['edge_njv'] for words in row for word in words])
    print 'njv', len(list_word), len(list_edge), list_edge.issubset(list_word)

 ./files/OpinosisDataset1.0/preprocessed/test/eyesight-issues_amazon_kindle.dump
all 437 433 True
nj 234 197 True
njv 315 314 True
./files/OpinosisDataset1.0/preprocessed/test/food_swissotel_chicago.dump
all 391 391 True
nj 244 226 True
njv 296 296 True
./files/OpinosisDataset1.0/preprocessed/test/speed_windows7.dump
all 546 542 True
nj 313 274 True
njv 396 391 True
./files/OpinosisDataset1.0/preprocessed/test/features_windows7.dump
all 414 411 True
nj 227 198 True
njv 305 302 True
./files/OpinosisDataset1.0/preprocessed/test/display_garmin_nuvi_255W_gps.dump
all 347 345 True
nj 195 167 True
njv 251 246 True
./files/OpinosisDataset1.0/preprocessed/test/interior_toyota_camry_2007.dump
all 527 524 True
nj 334 301 True
njv 410 405 True
./files/OpinosisDataset1.0/preprocessed/test/video_ipod_nano_8gb.dump
all 680 675 True
nj 440 385 True
njv 541 537 True
./files/OpinosisDataset1.0/preprocessed/test/rooms_bestwestern_hotel_sfo.dump
all 890 886 True
nj 545 490 True
njv 687 683 True
./files/O

In [23]:
dict_tmp = Filer.readdump('./files/OpinosisDataset1.0/preprocessed/test/accuracy_garmin_nuvi_255W_gps.dump')

In [44]:
filepath = glob.glob('./files/OpinosisDataset1.0/preprocessed/test/*')
for path in filepath:
    dict_tmp = Filer.readdump(path)
    if len(dict_tmp['sentence']) == 101:
        print path

./files/OpinosisDataset1.0/preprocessed/test/sound_ipod_nano_8gb.dump


### OpinosisDatasetの形態素解析
1. トピックごとにdictを作成する
 * sentence: 元の文のリスト
 * sep_all: 含まれているすべての単語のリスト
 * sep_nj: 含まれている名詞と形容詞のリスト
 * sep_njv: 含まれている名詞と形容詞と動詞のリスト
 * pos_all: 含まれているすべての単語のpos
 * pos_nj: 含まれている名詞と形容詞のpos
 * pos_njv: 含まれている名詞と形容詞と動詞のpos
 * edge_nj: すべての単語の構文によるエッジ
 * edge_nj: 名詞と形容詞の構文によるエッジ
 * edge_njv: 名詞と形容詞と動詞の構文によるエッジ

In [8]:
list_path = glob.glob('./files/OpinosisDataset1.0/topics/*')

In [9]:
proc = CoreNLP(configdict={'annotators': 'tokenize, ssplit, pos, lemma, parse'},
               corenlp_jars=["/home/ikegami/lib/stanford-corenlp-full-2015-04-20/*"])

for path in list_path:
    dict_total = {'sentence':[],
                  'sep_all':[],
                  'sep_nj':[],
                  'sep_njv':[],
                  'pos_all':[],
                  'pos_nj':[],
                  'pos_njv':[],
                  'edge_all': [],
                  'edge_nj':[],
                  'edge_njv': []}
    filename = path.replace('./files/OpinosisDataset1.0/topics/', '').replace('.txt.data', '')
    outputpath = './files/OpinosisDataset1.0/preprocessed3/'+filename+'.dump'
    list_corpus = Filer.readtxt(path, LF='\r\n')
    for corpus in list_corpus:
        try:
            list_sentence_a, list_bag_a, list_edge_a, list_pos_a = morpho_e_pos(corpus)
            list_sentence_e, list_bag_e, list_edge_e, list_pos_e = morpho_e_nj_pos(corpus)
            list_sentence_v, list_bag_v, list_edge_v, list_pos_v = morpho_e_njv_pos(corpus)
            dict_total['sentence'].extend(list_sentence_e)
            dict_total['sep_all'].extend(list_bag_a)
            dict_total['sep_nj'].extend(list_bag_e)
            dict_total['sep_njv'].extend(list_bag_v)
            dict_total['pos_all'].extend(list_pos_a)
            dict_total['pos_nj'].extend(list_pos_e)
            dict_total['pos_njv'].extend(list_pos_v)
            dict_total['edge_all'].extend(list_edge_a)
            dict_total['edge_nj'].extend(list_edge_e)
            dict_total['edge_njv'].extend(list_edge_v)
        except UnicodeDecodeError:
            print 'error'
    Filer.writedump(dict_total, outputpath)
    print outputpath

INFO:CoreNLP_PyWrapper:Starting java subprocess, and waiting for signal it's ready, with command: exec java -Xmx4g -XX:ParallelGCThreads=1 -cp '/home/ikegami/ikegami/lib/python2.7/site-packages/stanford_corenlp_pywrapper/lib/*:/home/ikegami/lib/stanford-corenlp-full-2015-04-20/*'      corenlp.SocketServer --outpipe /tmp/corenlp_pywrap_pipe_pypid=11416_time=1472539223.36  --configdict '{"annotators": "tokenize, ssplit, pos, lemma, parse"}'
INFO:CoreNLP_PyWrapper:Successful ping. The server has started.
INFO:CoreNLP_PyWrapper:Subprocess is ready.


./files/OpinosisDataset1.0/preprocessed3/gas_mileage_toyota_camry_2007.dump
error
error
error
error
error
./files/OpinosisDataset1.0/preprocessed3/location_bestwestern_hotel_sfo.dump
./files/OpinosisDataset1.0/preprocessed3/interior_honda_accord_2008.dump
./files/OpinosisDataset1.0/preprocessed3/sound_ipod_nano_8gb.dump
./files/OpinosisDataset1.0/preprocessed3/size_asus_netbook_1005ha.dump
./files/OpinosisDataset1.0/preprocessed3/eyesight-issues_amazon_kindle.dump
./files/OpinosisDataset1.0/preprocessed3/display_garmin_nuvi_255W_gps.dump
./files/OpinosisDataset1.0/preprocessed3/video_ipod_nano_8gb.dump
error
./files/OpinosisDataset1.0/preprocessed3/buttons_amazon_kindle.dump
./files/OpinosisDataset1.0/preprocessed3/screen_ipod_nano_8gb.dump
error
./files/OpinosisDataset1.0/preprocessed3/location_holiday_inn_london.dump
./files/OpinosisDataset1.0/preprocessed3/comfort_honda_accord_2008.dump
error
./files/OpinosisDataset1.0/preprocessed3/service_bestwestern_hotel_sfo.dump
./files/Opinosi

In [2]:
### 楽天市場のデータの集計

In [5]:
csv.field_size_limit(1000000000)

131072

In [6]:
# 商品数の集計
list_path = glob.glob('./files/rakuten_ichiba/rawfile/tsvfile/*')
list_path.sort()
list_product = []
for path in list_path:
    print path
    list_tmp = Filer.readtsv(path)
    list_tmp = [row[3] for row in list_tmp]
    list_product.append(list_tmp)

./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201001_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201002_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201003_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201004_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201005_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201006_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201007_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201008_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201009_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201010_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201011_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201012_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201101_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile

In [8]:
# 2012年の商品ごとのレコード数 => 2012年だけ考えれば十分
list_2012 = []
for row in list_product[24:]:
    list_2012.extend(row)

dict_count = collections.Counter(list_2012)
for row in sorted(dict_count.items(), key=lambda x:x[1], reverse=True)[0:20]:
    print row[0], row[1]

darkangel:312-535-01216 28257
donya:88584-ss 21796
soukai:9000009984074 16563
angfa:648226 14003
z-craft:1239 11524
low-ya:vg-groony 11443
tea-life:90199 11167
gourmetcoffee:2645 10228
shaddy:116961130set 10210
aimere:e0020014 9607
pierrot:a1202-012414 9373
kyunan:1418025 9349
morrymama:10000134 8946
hseason:865804 8872
oga:1011967 8785
oga:1437888 8456
kyunan:mirainokouso-premium 8059
aarti:10000089 7909
hobinavi:time4 7878
kumamoto-food:10000601 7710


In [9]:
# 多すぎても困るので、10位から40位までの30商品文のreviewを取得
for row in sorted(dict_count.items(), key=lambda x:x[1], reverse=True)[10:40]:
    print row[0], row[1]

pierrot:a1202-012414 9373
kyunan:1418025 9349
morrymama:10000134 8946
hseason:865804 8872
oga:1011967 8785
oga:1437888 8456
kyunan:mirainokouso-premium 8059
aarti:10000089 7909
hobinavi:time4 7878
kumamoto-food:10000601 7710
sawaicoffee-tea:saledripbag 7668
maxshare:a05750_sale 7626
nat-gar:diet_fastyplacenta 7529
lala-boutique:7380043 7330
z-craft:1239-0076 7222
arai:555329 7121
ecojiji:enelong_3_1 7065
seasky:ink-100en 6959
kira-kira:kmb0001 6949
low-ya:vg-pola 6893
darkangel:nctop-6670 6885
childhoodpepit:10000984 6843
netstar:leg002 6814
kyunan:20grain 6728
iloveheaven:wkic01 6689
bestmart:103368 6660
vour-voir:vb325 6623
freshveg:03-076 6517
minamiya-shop:945199 6413
apm24:mjsc40201n-time-100 6400


In [10]:
# 特定の商品を取得
list_products = ["pierrot:a1202-012414", "kyunan:1418025", "morrymama:10000134",
                 "hseason:865804", "oga:1011967", "oga:1437888",
                 "kyunan:mirainokouso-premium", "aarti:10000089", "hobinavi:time4",
                 "kumamoto-food:10000601", "sawaicoffee-tea:saledripbag", "maxshare:a05750_sale",
                 "nat-gar:diet_fastyplacenta", "lala-boutique:7380043", "z-craft:1239-0076",
                 "arai:555329", "ecojiji:enelong_3_1", "seasky:ink-100en",
                 "kira-kira:kmb0001", "low-ya:vg-pola", "darkangel:nctop-6670",
                 "childhoodpepit:10000984", "netstar:leg002", "kyunan:20grain",
                 "iloveheaven:wkic01", "bestmart:103368", "vour-voir:vb325",
                 "freshveg:03-076", "minamiya-shop:945199", "apm24:mjsc40201n-time-100"]

dict_products = {product:[] for product in list_products}

list_path = glob.glob('./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review2012*')
for path in list_path:
    print path
    list_tmp = Filer.readtsv(path)
    list_tmp = [row for row in list_tmp if row[3] in list_products]
    for row in list_tmp:
        dict_products[row[3]].append(row)

./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201206_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201202_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201208_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201203_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201205_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201210_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201204_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201211_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201207_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201209_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201201_20140221.tsv
./files/rakuten_ichiba/rawfile/tsvfile/ichiba04_review201212_20140221.tsv


In [12]:
# 商品ごとにtsvに保存する
for product in list_products:
    Filer.writetsv(dict_products[product],
                   './files/rakuten_ichiba/rawfile/productfile/'+product+'.tsv')

In [4]:
# reviewの部分のみ抜き出す
list_path = glob.glob('./files/rakuten_ichiba/rawfile/productfile/*')
for path in list_path:
    name = path.replace('./files/rakuten_ichiba/rawfile/productfile/', '')
    list_tmp = Filer.readtsv(path)
    list_tmp = [[row[15]] for row in list_tmp]
    Filer.writetsv(list_tmp, './files/rakuten_ichiba/processedfile/reviewfile/'+name)

In [38]:
# 文単位に分ける
list_path = glob.glob('./files/rakuten_ichiba/processedfile/reviewfile/*')
for path in list_path:
    name = path.replace('./files/rakuten_ichiba/processedfile/reviewfile/', '')
    list_tmp = Filer.readtsv(path)
    list_sentence = []
    # ビックリマークを句点に変える
    for sen in list_tmp:
        sentence = sen[0].replace('!', '。').replace('！', '。')
        list_sentence.extend(sentence.split('。'))
    list_sentence = [[row] for row in list_sentence if row != '']
    Filer.writetsv(list_sentence, './files/rakuten_ichiba/processedfile/sentencefile/'+name)

In [53]:
# 文を形態素解析して、list_dict_sentence_allword_wordを作る
list_path = glob.glob('./files/rakuten_ichiba/processedfile/sentencefile/*')
for num, path in enumerate(list_path):
    list_tmp = Filer.readtsv(path)
    list_master = []
    for sen in list_tmp:
        if len(sen) != 0:
            sentence = sen[0].strip()
            list_all_word, list_word = morpho_j(sentence)
            dict_sen_allword_word = {'sentence': sentence,
                                     'word_all': list_all_word,
                                     'word_nj': list_word}
            list_master.append(dict_sen_allword_word)
    Filer.writedump(list_master, './files/rakuten_ichiba/processedfile/morphofile/%s.dump' % num)

In [54]:
dict_tmp = Filer.readdump('./files/rakuten_ichiba/processedfile/morphofile/1.dump')

In [57]:
# 採用したクラスタをtsvとして吐き出す
list_cluster_num = [[0,1], [0,2], [0,4], [0,6], [1,1], 
                    [1,4], [1,5], [1,7], [2,2], [2,3],
                    [3,2], [3,3], [4,3], [5,1], [5,6],
                    [6,1], [6,5], [8,0], [8,4], [9,3],
                    [12,3], [12,5], [14,2], [14,6], [15,2],
                    [15,3], [20,2], [20,3], [20,5], [21,1],
                    [21,3], [21,7], [25,2], [25,3], [27,0],
                    [27,1], [27,2], [29,0], [29,3], [29,4]]

for num0, num1 in list_cluster_num:
    inputpath = './files/rakuten_ichiba/processedfile/classifiedfile/%s/cluster%s.dump' % (num0, num1)
    list_dict = Filer.readdump(inputpath)
    list_master = [[row['sentence']] for row in list_dict]
    Filer.writetsv(list_master,
                   './files/rakuten_ichiba/processedfile/classifiedfile/tsvfile/%s_%s.tsv'%(num0, num1))

In [1]:
### goldのチェック

In [6]:
dict_test = Filer.readdump('./files/OpinosisDataset1.0/preprocessed1/test/accuracy_garmin_nuvi_255W_gps.dump')

In [8]:
len(dict_test['sentence'])

67

In [3]:
dict_tmp = Filer.readdump('./files/OpinosisDataset1.0/preprocessed1/ans/accuracy_garmin_nuvi_255W_gps.dump')

In [5]:
dict_tmp['sentence']

[[u'This unit is generally quite accurate',
  u'Set-up and usage are considered to be very easy',
  u'The maps can be updated and tend to be reliable'],
 [u'Its accurate fast and its simple operations make this a for sure buy'],
 [u'It is very accurate even in destination time'],
 [u'The Garmin seems to be generally very accurate',
  u"It 's easy to use with an intuitive interface"],
 [u'Very accurate with travel and destination time',
  u'Negatives are not accurate with speed limits and rural roads']]

### アンケートで表示するようのデータを作成

In [94]:
list_filepath = ['../topic_model/files/rakuten_corpus/rakuten_corpus_master/txtfile/travel02_userReview05_20160304.txt',
                 '../topic_model/files/rakuten_corpus/rakuten_corpus_master/txtfile/travel02_userReview06_20160304.txt',]
list_fileeva = ['./files/rakuten/evaluation/txtfile/travel01_userEvaluation05_20160304.txt',
                './files/rakuten/evaluation/txtfile/travel01_userEvaluation06_20160304.txt',]

In [95]:
# 読み込み
list_file = [Filer.readtxt(path)  for path in list_filepath]
# タブで区切る
list_file = [sentence.split('\t') for row in list_file for sentence in row]

In [96]:
# 施設番号と年代で検索
hotel_num = '54127'
list_year = ['2010', '2011']
list_file_rev = [row[0:4] for row in list_file if row[0]==hotel_num and row[1][0:4] in list_year]

In [97]:
print len(list_file_rev)
list_eva = [Filer.readtxt(path) for path in list_fileeva]
list_eva = [sentence.split('\t') for row in list_eva for sentence in row]

1649


In [98]:
dict_eva = {row[0]:[row[2], row[7], row[8],
                    row[9], row[10], row[11],
                    row[12], row[13]] for row in list_eva}

list_review_eva = [row+dict_eva[row[3]] for row in list_file_rev]

In [29]:
Filer.writetsv(list_review_eva, './files/rakuten/txt_evaluation/txt_evaluation_1238.tsv')

In [99]:
list_review_eva[0]

['54127',
 '2010/01/01 15:24:28',
 '\xe3\x81\x8a\xe9\x83\xa8\xe5\xb1\x8b\xe3\x81\xae\xe3\x81\x8a\xe9\xa2\xa8\xe5\x91\x82\xe3\x81\x8c\xe3\x81\xa8\xe3\x81\xa6\xe3\x82\x82\xe5\xba\x83\xe3\x81\x8f\xe3\x81\xa6\xe5\xad\x90\xe4\xbe\x9b\xe3\x81\xa8\xe4\xb8\x80\xe7\xb7\x92\xe3\x81\xab\xe3\x82\x86\xe3\x81\xa3\xe3\x81\x8f\xe3\x82\x8a\xe5\x85\xa5\xe3\x82\x8c\xe3\x81\xa6\xe8\x89\xaf\xe3\x81\x8b\xe3\x81\xa3\xe3\x81\x9f\xe3\x81\xa7\xe3\x81\x99\xe3\x80\x82   \xe3\x80\x90\xe3\x81\x94\xe5\x88\xa9\xe7\x94\xa8\xe3\x81\xae\xe5\xae\xbf\xe6\xb3\x8a\xe3\x83\x97\xe3\x83\xa9\xe3\x83\xb3\xe3\x80\x91 \xe3\x80\x90\xe3\x81\x8a\xe6\x97\xa5\xe3\x81\xab\xe3\x81\xa1\xe9\x99\x90\xe5\xae\x9a\xef\xbc\x81\xe3\x80\x91\xe4\xbd\x8e\xe5\xb1\xa4\xe9\x9a\x8e\xe3\x81\xa0\xe3\x81\x8b\xe3\x82\x89\xe3\x81\x8a\xe5\xbe\x97\xef\xbc\x81\xe8\x88\x9e\xe6\xb5\x9c\xe3\x82\xb9\xe3\x83\x9a\xe3\x82\xb7\xe3\x83\xa3\xe3\x83\xab\xef\xbc\x88\xe3\x81\x8a\xe5\xbc\x81\xe5\xbd\x93\xe4\xbb\x98\xef\xbc\x89 \xe3\x81\x8a\xe9\x83\xa8\xe5\xb1\x8b\xe3\x82\xb

In [100]:
#【ご利用の宿泊プラン】~を削除
pattern = r'【ご利用の宿泊プラン】.+$'
list_review_eva1 = []
for row in list_review_eva:
    list_tmp = [row[3], row[4], row[1]]
    list_tmp.append(re.sub(pattern, '', row[2]))
    list_tmp = list_tmp + row[5:12]
    list_review_eva1.append(list_tmp)

In [43]:
Filer.writetsv(list_review_eva1, './files/rakuten/txt_evaluation/txt_evaluation_1238_rev.tsv')

In [101]:
# 読み込む形式に整形
from datetime import datetime as dt
list_review_eva1 = sorted(list_review_eva1, key=lambda x:x[2], reverse=True)
list_eva = [{'user_id':int(list_review_eva1[num][0]), 'user_name':list_review_eva1[num][1],
             'review':list_review_eva1[num][3], 'date_time':dt.strptime(list_review_eva1[num][2], '%Y/%m/%d %H:%M:%S'),
             'eva1':int(list_review_eva1[num][4]),'eva2':int(list_review_eva1[num][5]),
             'eva3':int(list_review_eva1[num][6]),'eva4':int(list_review_eva1[num][7]),
             'eva5':int(list_review_eva1[num][8]),'eva6':int(list_review_eva1[num][9]),
             'eva7':int(list_review_eva1[num][10])} for num in range(len(list_review_eva1))]

In [102]:
Filer.writedump(list_eva, './files/rakuten/txt_evaluation/txt_evaluation_54127.dump')

In [103]:
len(list_review_eva1)

1649

In [104]:
for i in range(1,8):
    counter = 0
    for row in list_review_eva1:
        counter+=int(row[3+i])
    print float(counter)/len(list_review_eva1)

4.50818677987
4.63311097635
2.70466949666
4.53365676167
4.56337174045
4.42207398423
4.61370527592


In [105]:
list_review = Filer.readdump('./files/rakuten/txt_evaluation/txt_evaluation_54127.dump')

In [106]:
len(list_review)

1649

In [107]:
list_all = []
for row in list_review:
    list_s = re.split(r'。|！', row['review'])
    list_s = [sen for sen in list_s if len(sen) > 1]
    for sen in list_s:
        list_word_all, list_word = morpho_j(sen)
        dict_tmp = {'sentence': sen,
                    'user_name': row['user_name'],
                    'date_time': row['date_time'],
                    'sep_nj': list_word,
                    'sep_all': list_word_all}
        list_all.append(dict_tmp)

In [108]:
Filer.writedump(list_all, './files/rakuten/txt_evaluation/txt_sep_54127.dump')

### 分類表の整形

In [111]:
list_csv = Filer.readcsv('./files/dictionary/bunruidb/bunruidb.txt')

In [118]:
list_csv = [[row[7],
             row[12].decode('shift-jis').encode('utf-8'),
             row[13].decode('shift-jis').encode('utf-8'),] for row in list_csv]

In [120]:
dict_word_class = {row[i]: row[0] for row in list_csv for i in range(1,3)}

In [124]:
Filer.writedump(dict_word_class, './files/dictionary/dict_word_class.dump')

### 極性辞書の作成

In [126]:
list_txt = Filer.readtxt('./files/dictionary/pn_ja.txt', LF='\r\n')
list_txt = [row.decode('shift-jis').encode('utf-8') for row in list_txt]

In [131]:
dict_word_pol = {}
for row in list_txt:
    list_row = row.split(':')
    dict_word_pol[list_row[0]] = float(list_row[3])
    if len(list_row[1].decode('utf-8')) > 1:
        dict_word_pol[list_row[1]] = float(list_row[3])
Filer.writedump(dict_word_pol, './files/dictionary/dict_word_pol.dump')

In [136]:
list_file = Filer.readdump('./files/rakuten/for_questionnaire/54127/txt_sep_54127.dump')

In [137]:
list_master = []
for row in list_file:
    dict_tmp = {}
    list_sep_all, list_sep, list_sep_all_lemmas, list_sep_lemmas = morpho_j(row['sentence'])
    dict_tmp['date_time'] = row['date_time']
    dict_tmp['sentence'] = row['sentence']
    dict_tmp['user_name'] = row['user_name']
    dict_tmp['sep_all'] = list_sep_all
    dict_tmp['sep_nj'] = list_sep
    dict_tmp['sep_all_lemmas'] = list_sep_all_lemmas
    dict_tmp['sep_nj_lemmas'] = list_sep_lemmas
    list_master.append(dict_tmp)
Filer.writedump(list_master, './files/rakuten/for_questionnaire/54127/txt_sep_54127.dump')